In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import seaborn as sns
# PyTorch
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler
import torch.nn as nn

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Data science tools
import numpy as np
import pandas as pd
import os

# Image manipulations
from PIL import Image
# Useful for examining network
# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 14

# Printing out all outputs

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode


In [ ]:
pneumonia_frame = pd.read_csv("data_metadata.csv")
n = 4000
img_name = pneumonia_frame.iloc[n, 1]
pneumonia = pneumonia_frame.iloc[n, 5:6]


print('Image name: {}'.format(img_name))
print('Landmarks shape: {}'.format(pneumonia.shape))
print('First 4 Landmarks: {}'.format(pneumonia))

In [ ]:
def show_pneumonia(image, pneumonia):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.pause(0.001) 
    
    
plt.figure()
show_pneumonia(io.imread(os.path.join('data/train/1', img_name),as_gray=True),
               pneumonia)
plt.show()# pause a bit so that plots are updated

In [ ]:
class PneumoniaDataset(Dataset):
    """Pneumonia dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.pneumonia_frame = pd.read_csv(csv_file)
        self.pneumonia_frame = pd.get_dummies(self.pneumonia_frame, columns=['Label'])
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.pneumonia_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.pneumonia_frame.iloc[idx, 1])
        image = io.imread(img_name, as_gray=False, pilmode="RGB")
        pneumonia = self.pneumonia_frame.iloc[idx, 5:6]
        pneumonia = np.array(float(pneumonia))
        sample = {'image': image, 'pneumonia': pneumonia}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
pneumonia_df = PneumoniaDataset(csv_file='data_metadata.csv',
                                    root_dir='data/train/1')

In [ ]:
pneumonia_df[0]

In [ ]:
fig = plt.figure()

for i in range(3139,3142):
    sample = pneumonia_df[i]
    print(i, sample['image'].shape, sample['pneumonia'])
    ax = plt.subplot(1, 5000, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_pneumonia(**sample)

In [ ]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, pneumonia = sample['image'], sample['pneumonia']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # h and w are swapped for landmarks because for images,
        # x and y axes are axis 1 and 0 respectively
        pneumonia = pneumonia

        return {'image': img, 'pneumonia': pneumonia}

In [ ]:
class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, pneumonia = sample['image'], sample['pneumonia']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        return {'image': image, 'pneumonia': pneumonia}

In [ ]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, pneumonia = sample['image'], sample['pneumonia']
        image = image.transpose((2, 0, 1))

        return {'image': torch.from_numpy(image),
                'pneumonia': torch.from_numpy(pneumonia)}

In [ ]:
transformed_dataset = PneumoniaDataset(csv_file='data_metadata.csv',
                                    root_dir='data/train/1',
                                           transform=transforms.Compose([
                                               Rescale(256),
                                               RandomCrop(224),
                                               ToTensor()
                                           ]))

In [ ]:
first_data = transformed_dataset[0]

features = first_data["image"]
labels = first_data["pneumonia"]
features,labels


In [ ]:
for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['image'].size(), sample['pneumonia'])

    if i == 10:
        break

In [ ]:
batch_size = 10
dataloader = DataLoader(transformed_dataset, batch_size = batch_size, shuffle = True)

In [ ]:
dataiter= iter(dataloader)
data = dataiter.next()
features = data["image"]
labels = data["pneumonia"]
print(features.shape,labels.shape, len(features), len(labels))